### Importing libraries ###

In [ ]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torchtext


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\bm2ic\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\bm2ic\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\bm2ic\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_lo

In [51]:
%pip install torchtext
from torchtext.vocab import GloVe;
global_vectors = GloVe(name='6B', dim=300);

Note: you may need to restart the kernel to use updated packages.


.vector_cache\glove.6B.zip: 862MB [03:09, 4.54MB/s]                               
100%|█████████▉| 399999/400000 [00:55<00:00, 7198.30it/s]


In [3]:
import re
import torchtext.vocab as tv


In [4]:
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import re
from argparse import Namespace
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
import sentencepiece as spm


In [1]:
from argparse import Namespace

args = Namespace(
data_directory = "../data/balanced_dataset.csv",
seed = 1300,
max_sentence_length = 100,
lower = True,
start_token = "<sos>",
end_token = "<eos>",
unk_token = "<unk>",
pad_token = "<pad>",
vocab_size = 0,
embedding_dim = 300,
device = "cuda" if torch.cuda.is_available() else "cpu",
);

NameError: name 'torch' is not defined

## DATA preprocessing ##
Done by Cynthia, Houmam, Abderraouf

In [6]:
toxic_frame = pd.read_csv(args.data_directory);
toxic_frame = toxic_frame.dropna()

toxic_frame

,Unnamed: 0,id,comment_text,target
0,0,0000997932d777bf,explanation why the edits made under my userna...,0.0
1,1,000103f0d9cfb60f,daww he matches this background colour im seem...,0.0
2,2,000113f07ec002fd,hey man im really not trying to edit war its j...,0.0
3,3,0001b41b1c6bb37e,more i cant make any real suggestions on impr...,0.0
4,4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0.0
...,...,...,...,...
2995,2995,08113e0533f6627a,new section at wpani there is now a new secti...,0.0
2996,2996,081166fea250a5af,and asking top stop involving me,0.0
2997,2997,08121cde0a727842,re all items i know that you said i did someth...,0.0
2998,2998,08143ca4834f8bcf,so you not going tell me why you created so m...,0.0


In [7]:
toxic_frame.isnull().sum()

Unnamed: 0      0
id              0
comment_text    0
target          0
dtype: int64

In [8]:
# Dictionary of English Contractions
import re
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}

# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
  def replace(match):
    return contractions_dict[match.group(0)]
  return contractions_re.sub(replace, text)

# Expanding Contractions in the reviews
toxic_frame['comment_text']= toxic_frame['comment_text'].apply(lambda x:expand_contractions(x));

In [9]:
import string


toxic_frame['comment_text'] = toxic_frame['comment_text'].apply(
    lambda x: re.sub(r'\w*\d\w*', '', x)
)
toxic_frame['comment_text']=toxic_frame['comment_text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
# Removing extra spaces
toxic_frame['comment_text']= toxic_frame['comment_text'].apply(lambda x: re.sub(' +',' ',x))
#turning all to lower case
toxic_frame['comment_text']= toxic_frame['comment_text'].apply(lambda x: x.lower())
for index,text in enumerate(toxic_frame['comment_text'][35:40]):
  print('Review %d:\n'%(index+1),text)

Review 1:
  not at all you are making a straw man argument here i never claimed odonohue had that position rather that practitioners and researchers in the field ignored the dsm position which is exactly what the quote says and also something odonohue agrees with again i was combating the notion that it was a absurd part to claim that pedophilia is a sexual orientation since many researchers hold this position it would be unfair to call it absurd the disorder part is divided in the field some argue that it is not a disorder at all some do at the end of the day it is a value judgment as cantor pointed out earlier in the thread not a scientific judgement if we choose to make this value judgment in the article it should be stated clearly and not pretend to have a scientific basis 
Review 2:
  mainland asia includes the lower basin of chinas yangtze river as well as korea but being specific is fine too i just found a citation for a more comprehensive dna study by hammer below rather than o

In [10]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [11]:
#removing stop words
stop_words = set(stopwords.words('english'))
toxic_frame['comment_text']= toxic_frame['comment_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

In [12]:
from nltk.stem import WordNetLemmatizer
import nltk

# Make sure you download required corpora
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bm2ic\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bm2ic\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\bm2ic\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [13]:
def lemmatize_text(text):
    tokens = nltk.word_tokenize(text)
    lemmatized = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(lemmatized)

# Apply to the column
toxic_frame['comment_text'] = toxic_frame['comment_text'].apply(lemmatize_text);


In [14]:
toxic_frame = toxic_frame.dropna()

toxic_frame['comment_text'] = toxic_frame['comment_text'].astype('string')


In [15]:
toxic_frame.isnull().sum()

Unnamed: 0      0
id              0
comment_text    0
target          0
dtype: int64

In [16]:
toxic_frame['comment_text'].dtype


string[python]

In [17]:
train_set_len = int(len(toxic_frame)*0.80);

test_set_len = int(len(toxic_frame)*0.10);

val_set_len = int(len(toxic_frame)*0.10);

toxic_frame.loc[:train_set_len,'split'] = "train";

toxic_frame.loc[train_set_len:train_set_len + test_set_len,'split'] = "test";

toxic_frame.loc[train_set_len + test_set_len:,'split'] = "val";

toxic_frame.to_csv(f"../data/data_split.csv");

args.data_split = "../data/data_split.csv";

In [18]:
import torch
from torch.utils.data import Dataset
def pad_sequence(tokenizer,encoded_ids, max_length, pad_id = args.pad_token):
    
    # Truncate if too long
    if len(encoded_ids) > max_length:
        encoded_ids = encoded_ids[:max_length]
    
    # Add padding
    padding = [pad_id] * (max_length - len(encoded_ids))
    return encoded_ids + padding;

In [26]:
class Vectorizer():
    
    def __init__(self,tokenizer,vocabulary):
        self.tokenizer = tokenizer;
        self.vocabulary = vocabulary;
        pass;
    
    @classmethod
    def from_dataframe(cls, csvFile,columns):
        """
        Instantiate the vectorizer from the dataset dataframe
        
        Args:

        dataFrame (pandas.DataFrame): Dataset 
        xColumnName : name of features column.

        Returns: an instance of the Vectorizer

        """

        datafile = pd.read_csv(csvFile);

        tokens_iterator = [];

        tokenizer = torchtext.data.utils.get_tokenizer("basic_english");
        
        for i,row in datafile[[columns[0]]].iterrows():
            text = row[columns[0]];
            # print(type(text));
            text = str(text)
            tokens = tokenizer(text);
            tokens_iterator.append(tokens);

        caption_vocab = tv.build_vocab_from_iterator(tokens_iterator,min_freq = 3,specials=[args.pad_token, args.unk_token, args.start_token , args.end_token]);
        
        caption_vocab.set_default_index(caption_vocab[args.unk_token]);
   
        return cls(tokenizer,caption_vocab);
    
    
    
    def vectorize(self,sentence , max_length , add_special = False):
        
        """
        Create a numerical vector for the sentence
        
        Args:
        tokenizer: sentencepiece tokenizer.
        sentence (str): the sentence
        add_special (boolean) : Adding special tokens (<sos>,<eos>).
        
        Returns:
        Tokens indices vector.
        
        """
        vector = self.tokenizer(sentence);
        vector = pad_sequence(self.tokenizer,vector, max_length);
        ids = [];
        for i,token in enumerate(vector):
            ids.append(self.vocabulary[token]);
        
        return ids;


In [27]:
class CustomDataSet(Dataset):
    
    def __init__(self,datafile,vectorizer):
        
        dataFrame = pd.read_csv(datafile);
        
        self._dataFrame = dataFrame;
        self._vectorizer = vectorizer;
        
        self.train_df = self._dataFrame[self._dataFrame.split == "train"];
        self.train_size = len(self.train_df);
        
        self.val_df = self._dataFrame[self._dataFrame.split == "val"];
        self.val_size = len(self.val_df);
        
        self.test_df = self._dataFrame[self._dataFrame.split == "test"];
        self.test_size = len(self.test_df);

        self._lookup_dict = {'train': (self.train_df, self.train_size),
        'val': (self.val_df, self.val_size),
        'test': (self.test_df, self.test_size)};
        
        self.set_split();  # setting train as default data set.

    
    @classmethod
    def load_dataset_and_make_vectorizer(cls,datafile):
        datafile = datafile;
        return cls(datafile, Vectorizer.from_dataframe(datafile,['comment_text']));

    
    def get_vectorizer(self):
        return self._vectorizer;
    
    
    def set_split(self, split="train"):
        
        """ selects the splits in the dataset using a column in the dataframe
        Args:finaldata.csv
        split (str): one of "train", "val", or "test"
        """
        self._target_split = split;

        self._target_df, self._target_size = self._lookup_dict[split];
    
    

    def __getitem__(self, index):
        
        """
        the primary entry point method for PyTorch datasets
        Args:
        index (int): the index to the data point
        Returns:
        image,target tensors + image name for testing purpose.
        """
        row = self._target_df.iloc[index];        
        
        source = self._vectorizer.vectorize(row['comment_text'],args.max_sentence_length);
        
        target = row['target'];

        return torch.tensor(source),torch.tensor(target);
    
    def __len__(self):
        return self._target_size;


    

In [28]:
dataset = CustomDataSet.load_dataset_and_make_vectorizer(args.data_split)
dataloader = DataLoader(dataset,batch_size=20);


In [22]:
toxic_frame['comment_text'].dtype


string[python]

In [108]:
for x,y in dataloader:
    break;
print(x,y)

tensor([[ 660,   81,    4,  ...,    0,    0,    0],
        [   1,   59, 1472,  ...,    0,    0,    0],
        [ 525,  383,   70,  ...,    0,    0,    0],
        ...,
        [ 432,    1, 2908,  ...,    0,    0,    0],
        [   4,    1,  118,  ...,    0,    0,    0],
        [  57,  233,    5,  ...,    0,    0,    0]]) tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
        0., 0.])


In [30]:
import numpy as np

In [31]:
import torch
import torch.nn as nn

def prepare_embedding_matrix(dataset_words, pre_embeddings):
    word_to_idx = pre_embeddings.stoi
    embedding_matrix = pre_embeddings.vectors
    embedding_dim = embedding_matrix.shape[1]

    final_embeddings = torch.zeros((len(dataset_words), embedding_dim))

    for i, word in enumerate(dataset_words):
        if word in word_to_idx and word_to_idx[word] < embedding_matrix.shape[0]:
            final_embeddings[i] = embedding_matrix[word_to_idx[word]]
        else:
            embedding_i = torch.empty(embedding_dim)
            torch.nn.init.xavier_uniform_(embedding_i.unsqueeze(0))
            final_embeddings[i] = embedding_i

    embedding_layer = nn.Embedding(
        num_embeddings=len(dataset_words),
        embedding_dim=embedding_dim
    )
    embedding_layer.weight.data = final_embeddings

    return embedding_layer


In [32]:
embedding_layer = prepare_embedding_matrix(list(dataset.get_vectorizer().vocabulary.get_stoi().keys()),global_vectors);

NameError: name 'global_vectors' is not defined

In [ ]:
for i in range(5):
